In [10]:
import sys  
sys.path.append( '..')

In [11]:
from datetime import datetime, timedelta
import pandas as pd


In [12]:
from src.stocks import *

In [13]:
# Define the stock codes you are interested in
stock_codes = ["RELIANCE.NS", "TCS.NS","DABUR.NS","ADANIGREEN.NS","SOLARA.NS","ZYDUSLIFE.NS","TATAMOTORS.NS","KPIL.NS"]
end_date = datetime.now() - timedelta(days=1)
start_date = end_date - timedelta(days=365)
start_date, end_date

(datetime.datetime(2023, 3, 12, 12, 57, 59, 370066),
 datetime.datetime(2024, 3, 11, 12, 57, 59, 370066))

In [14]:
generate_all_features(stock_codes, start_date, end_date)

generating features for  RELIANCE.NS


[*********************100%%**********************]  1 of 1 completed
/Users/ravinarukulla/PycharmProjects/stock_forecast/venv/lib/python3.9/site-packages/ta/trend.py:1030: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  self._psar[i] = high2
[*********************100%%**********************]  1 of 1 completed

file names:  features_RELIANCE.NS.csv target_RELIANCE.NS.csv features_RELIANCE.NS_future.csv
generating features for  TCS.NS



/Users/ravinarukulla/PycharmProjects/stock_forecast/venv/lib/python3.9/site-packages/ta/trend.py:1030: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  self._psar[i] = high2


file names:  features_TCS.NS.csv target_TCS.NS.csv features_TCS.NS_future.csv
generating features for  DABUR.NS


...

[*********************100%%**********************]  1 of 1 completed
/Users/ravinarukulla/PycharmProjects/stock_forecast/venv/lib/python3.9/site-packages/ta/trend.py:1030: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  self._psar[i] = high2


file names:  features_TATAMOTORS.NS.csv target_TATAMOTORS.NS.csv features_TATAMOTORS.NS_future.csv
generating features for  KPIL.NS


[*********************100%%**********************]  1 of 1 completed

file names:  features_KPIL.NS.csv target_KPIL.NS.csv features_KPIL.NS_future.csv



/Users/ravinarukulla/PycharmProjects/stock_forecast/venv/lib/python3.9/site-packages/ta/trend.py:1030: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  self._psar[i] = high2


In [15]:
forecast_df = build_n_forecast(stock_codes)

/Users/ravinarukulla/PycharmProjects/stock_forecast/venv/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [12:58:02] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


Building model for  RELIANCE.NS
mean Square error in train data: 2.5974433930193793e-06
Building model for  TCS.NS


/Users/ravinarukulla/PycharmProjects/stock_forecast/venv/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [12:58:03] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


mean Square error in train data: 1.0762058320592662e-05
Building model for  DABUR.NS


/Users/ravinarukulla/PycharmProjects/stock_forecast/venv/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [12:58:03] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


...

mean Square error in train data: 1.2963514042161638e-06
Building model for  TATAMOTORS.NS


/Users/ravinarukulla/PycharmProjects/stock_forecast/venv/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [12:58:06] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


mean Square error in train data: 2.3641967077201438e-06
Building model for  KPIL.NS


/Users/ravinarukulla/PycharmProjects/stock_forecast/venv/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [12:58:06] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


mean Square error in train data: 3.008536120362132e-07


In [16]:
forecast_df

,Date,forecast,Stock,Forecast_period
0,2024-03-11,2966.885010,RELIANCE.NS,2024-03-12
0,2024-03-11,4113.288574,TCS.NS,2024-03-12
0,2024-03-11,533.746582,DABUR.NS,2024-03-12
0,2024-03-11,1942.206421,ADANIGREEN.NS,2024-03-12
0,2024-03-11,425.746185,SOLARA.NS,2024-03-12
0,2024-03-11,993.877441,ZYDUSLIFE.NS,2024-03-12
0,2024-03-11,1029.214478,TATAMOTORS.NS,2024-03-12
0,2024-03-11,1073.246216,KPIL.NS,2024-03-12
